In [1]:
# Importações Essenciais
!pip install numpy # Install specific NumPy version
!pip install scikit-learn
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime
import warnings

In [2]:
# Imports do PyCaret
!pip install --upgrade pycaret      # Upgrade PyCaret to latest version
!pip install pycaret
from pycaret.classification import *

In [3]:
# Configurações do Pandas e Warnings
pd.set_option('display.max_columns', None) # Mostrar todas as colunas
warnings.filterwarnings("ignore") # Ignorar avisos

# Etapa 1: Carregamento dos Dados

In [4]:
print("="*30)
print("ETAPA 1: CARREGAMENTO DOS DADOS")
print("="*30)
print("Carregando dados...")

ETAPA 1: CARREGAMENTO DOS DADOS
Carregando dados...


In [5]:
# Tentar carregar os arquivos CSV
# Presume-se que os arquivos estão no mesmo diretório do notebook ou foram carregados no Colab
try:
    df_treino_completo = pd.read_csv('data.csv', encoding='utf-8')
    df_teste_final = pd.read_csv('Xtest.csv', encoding='utf-8') # Dados para previsão final (sem a coluna alvo)
    print("Dados carregados com sucesso.")
    print("Shape Dados de Treino:", df_treino_completo.shape)
    print("Shape Dados de Teste Final (Xtest.csv):", df_teste_final.shape)
except FileNotFoundError:
    print("\nErro: Arquivo(s) CSV não encontrado(s).")
    print("Certifique-se de que 'data.csv' e 'Xtest.csv' estão no ambiente do Colab.")
    print("Você pode carregá-los usando o painel 'Arquivos' à esquerda.")
    # Interromper a execução se os arquivos não forem encontrados
    exit()

Dados carregados com sucesso.
Shape Dados de Treino: (2240, 29)
Shape Dados de Teste Final (Xtest.csv): (110, 25)


In [6]:
# Exibir as primeiras linhas dos dados de treino
print("\nCabeçalho dos Dados de Treino:")
print(df_treino_completo.head())
print("\n" + "="*60 + "\n")


Cabeçalho dos Dados de Treino:
     ID  Year_Birth   Education Marital_Status   Income  Kidhome  Teenhome  \
0  5524        1957  Graduation         Single  58138.0        0         0   
1  2174        1954  Graduation         Single  46344.0        1         1   
2  4141        1965  Graduation       Together  71613.0        0         0   
3  6182        1984  Graduation       Together  26646.0        1         0   
4  5324        1981         PhD        Married  58293.0        1         0   

  Dt_Customer  Recency  MntWines  MntFruits  MntMeatProducts  MntFishProducts  \
0  2012-09-04       58       635         88              546              172   
1  2014-03-08       38        11          1                6                2   
2  2013-08-21       26       426         49              127              111   
3  2014-02-10       26        11          4               20               10   
4  2014-01-19       94       173         43              118               46   

   MntSweetP

# Etapa 2: Limpeza e Pré-processamento dos Dados (Aplicado aos Dados de Treino)

In [7]:
print("="*30)
print("ETAPA 2: LIMPEZA E PRÉ-PROCESSAMENTO (DADOS DE TREINO)")
print("="*30)

ETAPA 2: LIMPEZA E PRÉ-PROCESSAMENTO (DADOS DE TREINO)


In [8]:
# Criar uma cópia para limpeza
df_limpo = df_treino_completo.copy()

In [9]:
# --- 2.1 Remover Colunas Irrelevantes e Constantes ---
print("Removendo colunas ID, Z_CostContact, Z_Revenue...")
colunas_para_remover = ['ID', 'Z_CostContact', 'Z_Revenue']
df_limpo.drop(colunas_para_remover, axis=1, inplace=True, errors='ignore')
print(f"Colunas removidas. Shape atual: {df_limpo.shape}")

Removendo colunas ID, Z_CostContact, Z_Revenue...
Colunas removidas. Shape atual: (2240, 26)


In [10]:
# --- 2.2 Tratar Valores Ausentes (NaN) na Coluna 'Income' ---
print(f"\nLinhas iniciais: {df_limpo.shape[0]}")
print(f"Número de NaNs em 'Income': {df_limpo['Income'].isna().sum()}")
# Estratégia: Remover linhas com NaN em 'Income', pois representam uma pequena porcentagem
# e, com base na análise de referência, são majoritariamente de não-respondentes.
df_limpo.dropna(subset=['Income'], axis=0, inplace=True)
print(f"Linhas após remover NaNs em 'Income': {df_limpo.shape[0]}")
print("NaNs tratados.")


Linhas iniciais: 2240
Número de NaNs em 'Income': 24
Linhas após remover NaNs em 'Income': 2216
NaNs tratados.


In [11]:
# --- 2.3 Corrigir Inconsistências em 'Marital_Status' ---
print("\nCorrigindo valores inconsistentes em 'Marital_Status'...")
# Agrupar categorias raras/ambíguas em 'Single' (ou outra categoria apropriada)
df_limpo['Marital_Status'] = df_limpo['Marital_Status'].replace({'Alone': 'Single', 'Absurd': 'Single', 'YOLO': 'Single'})
print("Valores únicos de 'Marital_Status' após correção:", df_limpo['Marital_Status'].unique())


Corrigindo valores inconsistentes em 'Marital_Status'...
Valores únicos de 'Marital_Status' após correção: ['Single' 'Together' 'Married' 'Divorced' 'Widow']


In [12]:
# --- 2.4 Ajustar Tipos de Dados ---
print("\nAjustando tipos de dados...")
df_limpo = df_limpo.convert_dtypes() # Tenta inferir os melhores tipos possíveis (Int64, string, etc.)


Ajustando tipos de dados...


In [13]:
# Ajustes específicos
df_limpo['Dt_Customer'] = pd.to_datetime(df_limpo['Dt_Customer'])
df_limpo['Response'] = df_limpo['Response'].astype('int') # PyCaret prefere target como int ou string
df_limpo['Income'] = df_limpo['Income'].astype('float64') # Garantir que Income seja float para cálculos
print("Tipos de dados ajustados.")
print("\nInformações dos Dados Limpos:")
df_limpo.info()
print("\n" + "="*60 + "\n")

Tipos de dados ajustados.

Informações dos Dados Limpos:
<class 'pandas.core.frame.DataFrame'>
Index: 2216 entries, 0 to 2239
Data columns (total 26 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   Year_Birth           2216 non-null   Int64         
 1   Education            2216 non-null   string        
 2   Marital_Status       2216 non-null   string        
 3   Income               2216 non-null   float64       
 4   Kidhome              2216 non-null   Int64         
 5   Teenhome             2216 non-null   Int64         
 6   Dt_Customer          2216 non-null   datetime64[ns]
 7   Recency              2216 non-null   Int64         
 8   MntWines             2216 non-null   Int64         
 9   MntFruits            2216 non-null   Int64         
 10  MntMeatProducts      2216 non-null   Int64         
 11  MntFishProducts      2216 non-null   Int64         
 12  MntSweetProducts     2216 non-null   I

# Etapa 3: Engenharia de Atributos (Aplicada aos Dados de Treino)

In [14]:
print("="*30)
print("ETAPA 3: ENGENHARIA DE ATRIBUTOS (DADOS DE TREINO)")
print("="*30)

ETAPA 3: ENGENHARIA DE ATRIBUTOS (DADOS DE TREINO)


In [15]:
# Criar uma cópia para engenharia de atributos
df_eng = df_limpo.copy()

In [16]:
# --- 3.1 Calcular Idade ---
print("Calculando 'Age' a partir de 'Year_Birth'...")
ano_atual = datetime.now().year
df_eng['Age'] = ano_atual - df_eng['Year_Birth']
# Remover coluna original
df_eng.drop('Year_Birth', axis=1, inplace=True, errors='ignore')
print("'Age' calculada, 'Year_Birth' removida.")

Calculando 'Age' a partir de 'Year_Birth'...
'Age' calculada, 'Year_Birth' removida.


In [17]:
# --- 3.2 Calcular Tempo como Cliente (em anos) ---
# Usar a data máxima do dataset como referência para reprodutibilidade
print("\nCalculando 'Time_Customer' (em anos) a partir de 'Dt_Customer'...")
data_maxima = df_eng['Dt_Customer'].max()
df_eng['Time_Customer'] = (data_maxima - df_eng['Dt_Customer']).dt.days / 365.25
# Remover coluna original
df_eng.drop('Dt_Customer', axis=1, inplace=True, errors='ignore')
print("'Time_Customer' calculado, 'Dt_Customer' removida.")


Calculando 'Time_Customer' (em anos) a partir de 'Dt_Customer'...
'Time_Customer' calculado, 'Dt_Customer' removida.


In [18]:
# --- 3.3 Criar Novas Features ---
print("\nCriando novas features...")
# Total de filhos/adolescentes em casa
df_eng['Children_Total'] = df_eng['Kidhome'] + df_eng['Teenhome']
print("- Feature 'Children_Total' criada.")


Criando novas features...
- Feature 'Children_Total' criada.


In [19]:
# Gasto total em produtos
colunas_mnt = [col for col in df_eng.columns if 'Mnt' in col]
df_eng['Total_Spending'] = df_eng[colunas_mnt].sum(axis=1)
print("- Feature 'Total_Spending' criada.")

- Feature 'Total_Spending' criada.


In [20]:
# Total de compras realizadas
colunas_purch = [col for col in df_eng.columns if 'Purchases' in col]
df_eng['Total_Purchases'] = df_eng[colunas_purch].sum(axis=1)
print("- Feature 'Total_Purchases' criada.")

- Feature 'Total_Purchases' criada.


In [21]:
# Proporção de aceitação de campanhas anteriores (útil se houver muitas campanhas)
# Exemplo: Soma das AcceptedCmp1 a AcceptedCmp5
cmp_cols = ['AcceptedCmp1', 'AcceptedCmp2', 'AcceptedCmp3', 'AcceptedCmp4', 'AcceptedCmp5']
df_eng['Total_Acc_Cmp'] = df_eng[cmp_cols].sum(axis=1)
print("- Feature 'Total_Acc_Cmp' criada.")

- Feature 'Total_Acc_Cmp' criada.


In [22]:
# Renda per capita (considerando status marital para estimar tamanho da família - simplificado)
# Mapeamento simplificado: Single/Divorced/Widow -> 1; Together/Married -> 2
# Poderia ser mais complexo considerando filhos, mas vamos manter simples
def family_size_approx(status):
    if status in ['Single', 'Divorced', 'Widow']:
        return 1
    else: # Together, Married
        return 2

df_eng['Family_Size_Approx'] = df_eng['Marital_Status'].apply(family_size_approx)
df_eng['Income_Per_Capita'] = df_eng['Income'] / (df_eng['Family_Size_Approx'] + df_eng['Children_Total'])
df_eng['Income_Per_Capita'].replace([np.inf, -np.inf], 0, inplace=True) # Tratar divisão por zero se não houver ninguém na casa
df_eng['Income_Per_Capita'].fillna(0, inplace=True) # Tratar NaNs restantes
df_eng.drop('Family_Size_Approx', axis=1, inplace=True) # Remover coluna auxiliar
print("- Feature 'Income_Per_Capita' criada.")

- Feature 'Income_Per_Capita' criada.


In [23]:
# Reordenar colunas (foi opcional, coloca o alvo 'Response' no final)
colunas_principais = [col for col in df_eng.columns if col != 'Response']
df_final_features = df_eng[colunas_principais + ['Response']]

In [24]:
print("\nNovas features criadas com sucesso.")
print("\nCabeçalho dos Dados com Novas Features:")
print(df_final_features.head())
print("Shape dos Dados com Novas Features:", df_final_features.shape)
print("\n" + "="*60 + "\n")


Novas features criadas com sucesso.

Cabeçalho dos Dados com Novas Features:
    Education Marital_Status   Income  Kidhome  Teenhome  Recency  MntWines  \
0  Graduation         Single  58138.0        0         0       58       635   
1  Graduation         Single  46344.0        1         1       38        11   
2  Graduation       Together  71613.0        0         0       26       426   
3  Graduation       Together  26646.0        1         0       26        11   
4         PhD        Married  58293.0        1         0       94       173   

   MntFruits  MntMeatProducts  MntFishProducts  MntSweetProducts  \
0         88              546              172                88   
1          1                6                2                 1   
2         49              127              111                21   
3          4               20               10                 3   
4         43              118               46                27   

   MntGoldProds  NumDealsPurchases  Nu

# Etapa 4: Divisão Treino/Validação

In [25]:
print("="*30)
print("ETAPA 4: DIVISÃO TREINO/VALIDAÇÃO")
print("="*30)

ETAPA 4: DIVISÃO TREINO/VALIDAÇÃO


In [26]:
# Dividir os dados em treino (95%) e validação (5%) para avaliar o modelo final
# O conjunto de validação (df_validacao) não será visto durante o treinamento e tuning
df_treino = df_final_features.sample(frac=0.95, random_state=123)
df_validacao = df_final_features.drop(df_treino.index)

In [27]:
# Resetar os índices
df_treino.reset_index(inplace=True, drop=True)
df_validacao.reset_index(inplace=True, drop=True)

In [28]:
print(f'Dados para Modelagem (Treino): {df_treino.shape}')
print(f'Dados Não Vistos para Predição (Validação): {df_validacao.shape}')
print("\n" + "="*60 + "\n")

Dados para Modelagem (Treino): (2105, 31)
Dados Não Vistos para Predição (Validação): (111, 31)




# Etapa 5: Configuração do PyCaret

In [29]:
print("="*30)
print("ETAPA 5: CONFIGURAÇÃO DO PYCARET")
print("="*30)

ETAPA 5: CONFIGURAÇÃO DO PYCARET


In [30]:
# Definir explicitamente as features categóricas e numéricas
# (PyCaret pode inferir, mas é boa prática definir para evitar erros)
features_categoricas = ['Education', 'Marital_Status']
features_numericas = [col for col in df_treino.columns if df_treino[col].dtype in ['int64', 'float64', 'Int64', 'Float64'] and col not in features_categoricas + ['Response']]

In [31]:
print("Iniciando setup do PyCaret...")
# Configurar o ambiente do PyCaret para classificação
# Usaremos df_treino para treinar e comparar modelos
clf_setup = setup(
    data=df_treino,                 # Dados de treino (95% do original)
    target='Response',              # Coluna alvo
    session_id=123,                 # Para reprodutibilidade
    fix_imbalance=True,             # Lidar com desbalanceamento de classe (usando SMOTE por padrão)
    remove_outliers=True,           # Remover outliers (usando Isolation Forest por padrão)
    outliers_threshold=0.05,        # Limiar padrão para outliers
    normalize=True,                 # Normalizar features numéricas
    normalize_method='zscore',      # Método de normalização Z-score
    categorical_features=features_categoricas, # Features categóricas definidas
    numeric_features=features_numericas,       # Features numéricas definidas
    preprocess=True,                # Habilitar pré-processamento automático do PyCaret
                                    # (inclui imputação, one-hot encoding, etc.)
    # fold_strategy='stratifiedkfold', # Estratégia de validação cruzada (padrão para classificação)
    # fold=10,                       # Número de folds (padrão)
    verbose=True                   # Mostrar detalhes do setup
)
print("\nSetup do PyCaret concluído.")
print("\n" + "="*60 + "\n")

Iniciando setup do PyCaret...


,Description,Value
0,Session id,123
1,Target,Response
2,Target type,Binary
3,Original data shape,"(2105, 31)"
4,Transformed data shape,"(3086, 39)"
5,Transformed train set shape,"(2454, 39)"
6,Transformed test set shape,"(632, 39)"
7,Numeric features,28
8,Categorical features,2
9,Preprocess,True



Setup do PyCaret concluído.




# Etapa 6: Comparação de Modelos

In [32]:
print("="*30)
print("ETAPA 6: COMPARAÇÃO DE MODELOS")
print("="*30)

ETAPA 6: COMPARAÇÃO DE MODELOS


In [33]:
print("Comparando modelos com base na AUC (usando validação cruzada)...")
# Comparar diversos modelos usando validação cruzada e ordenar por AUC
# Isso dará uma ideia dos modelos mais promissores para este dataset
melhor_modelo_comparacao = compare_models(sort='AUC')

print("\nMelhor Modelo encontrado na comparação inicial:")
print(melhor_modelo_comparacao)
print("\n" + "="*60 + "\n")

Comparando modelos com base na AUC (usando validação cruzada)...


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
lightgbm,Light Gradient Boosting Machine,0.8934,0.9020,0.4757,0.7183,0.5679,0.5109,0.5271,1.7420
ridge,Ridge Classifier,0.8120,0.8983,0.8243,0.4354,0.5683,0.4629,0.5026,0.2340
lda,Linear Discriminant Analysis,0.8120,0.8982,0.8243,0.4354,0.5683,0.4629,0.5026,0.2420
lr,Logistic Regression,0.8214,0.8950,0.8243,0.4498,0.5807,0.4798,0.5164,0.8650
rf,Random Forest Classifier,0.8832,0.8899,0.3800,0.7139,0.4933,0.4344,0.4633,0.8060
xgboost,Extreme Gradient Boosting,0.8873,0.8897,0.4621,0.6878,0.5492,0.4882,0.5026,0.6550
et,Extra Trees Classifier,0.8894,0.8869,0.4073,0.7346,0.5184,0.4629,0.4910,0.4900
gbc,Gradient Boosting Classifier,0.8914,0.8810,0.5069,0.6935,0.5804,0.5204,0.5315,1.8680
svm,SVM - Linear Kernel,0.7963,0.8705,0.7789,0.4154,0.5381,0.4249,0.4614,0.2730
ada,Ada Boost Classifier,0.8601,0.8385,0.5338,0.5365,0.5326,0.4507,0.4522,0.5830


Processing:   0%|          | 0/65 [00:00<?, ?it/s]


Melhor Modelo encontrado na comparação inicial:
LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.1, max_depth=-1,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
               random_state=123, reg_alpha=0.0, reg_lambda=0.0, subsample=1.0,
               subsample_for_bin=200000, subsample_freq=0)




# Etapa 7: Tuning de Hiperparâmetros

In [34]:
print("="*30)
print("ETAPA 7: TUNING DE HIPERPARÂMETROS")
print("="*30)

ETAPA 7: TUNING DE HIPERPARÂMETROS


In [35]:
# Realizar tuning nos modelos mais promissores identificados na etapa anterior
# Otimizar pela métrica AUC

# --- Tuning do Melhor Modelo da Comparação ---
print(f"\nRealizando tuning do melhor modelo da comparação ({type(melhor_modelo_comparacao).__name__})...")
try:
    modelo_tunado_melhor = tune_model(melhor_modelo_comparacao, optimize='AUC', verbose=False)
    print("Tuning do melhor modelo concluído.")
    print(modelo_tunado_melhor)
except Exception as e:
    print(f"Erro ao fazer tuning do {type(melhor_modelo_comparacao).__name__}: {e}")
    modelo_tunado_melhor = melhor_modelo_comparacao # Usar o modelo original se o tuning falhar


Realizando tuning do melhor modelo da comparação (LGBMClassifier)...
Tuning do melhor modelo concluído.
LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.1, max_depth=-1,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
               random_state=123, reg_alpha=0.0, reg_lambda=0.0, subsample=1.0,
               subsample_for_bin=200000, subsample_freq=0)


In [ ]:
# --- Tuning de Outros Modelos Promissores (Exemplos) ---
# Pode-se adicionar mais modelos aqui se a comparação inicial indicar outros bons candidatos

# Exemplo: Tuning LightGBM (geralmente performa bem)
print("\nRealizando tuning do LightGBM...")
try:
    lgbm = create_model('lightgbm', verbose=False) # Criar o modelo primeiro
    modelo_tunado_lgbm = tune_model(lgbm, optimize='AUC', verbose=False)
    print("Tuning do LightGBM concluído.")
    print(modelo_tunado_lgbm)
except Exception as e:
    print(f"Erro ao fazer tuning do LightGBM: {e}")
    modelo_tunado_lgbm = None # Marcar como None se falhar


Realizando tuning do LightGBM...


In [ ]:
# Exemplo: Tuning XGBoost
print("\nRealizando tuning do XGBoost...")
try:
    xgboost = create_model('xgboost', verbose=False)
    modelo_tunado_xgboost = tune_model(xgboost, optimize='AUC', verbose=False)
    print("Tuning do XGBoost concluído.")
    print(modelo_tunado_xgboost)
except Exception as e:
    print(f"Erro ao fazer tuning do XGBoost: {e}")
    modelo_tunado_xgboost = None

print("\n" + "="*60 + "\n")


# Etapa 8: Blending de Modelos


In [ ]:
print("="*30)
print("ETAPA 8: BLENDING DE MODELOS")
print("="*30)

In [ ]:
# Combinar as previsões dos melhores modelos tunados pode melhorar a performance

# Selecionar os modelos tunados para o blend (ajuste conforme resultados do tuning)
modelos_para_blend = [m for m in [modelo_tunado_melhor, modelo_tunado_lgbm, modelo_tunado_xgboost] if m is not None]

modelo_blend = None
if len(modelos_para_blend) >= 2:
    print(f"Realizando blend de {len(modelos_para_blend)} modelos...")
    try:
        # Usar os top 2 ou 3 melhores tunados
        modelo_blend = blend_models(estimator_list=modelos_para_blend[:3], optimize='AUC', verbose=False)
        print("Blend de modelos concluído.")
        print(modelo_blend)
    except Exception as e:
        print(f"Erro ao realizar o blend: {e}. Usaremos o melhor modelo tunado individualmente.")
        modelo_blend = None # Resetar se o blend falhar
else:
    print("Não há modelos suficientes tunados com sucesso para realizar o blend.")

print("\n" + "="*60 + "\n")

# Etapa 9: Seleção e Finalização do Modelo Final

In [ ]:
print("="*30)
print("ETAPA 9: SELEÇÃO E FINALIZAÇÃO DO MODELO")
print("="*30)

In [ ]:
# Selecionar o melhor modelo entre o blend (se bem-sucedido) e os tunados individuais
# **Nota:** A seleção rigorosa requer comparar as métricas de validação cruzada (AUC) impressas
# nas etapas anteriores (compare_models, tune_model, blend_models).
# Para este exemplo, vamos priorizar o blend se ele foi criado, caso contrário, o melhor tunado.

modelo_final_escolhido = None
if modelo_blend:
    print("Selecionando o modelo Blended como modelo final.")
    modelo_final_escolhido = modelo_blend
else:
    print("Selecionando o melhor modelo tunado individualmente como modelo final (verifique AUCs acima).")
    # Aqui, você precisaria verificar qual dos 'modelo_tunado_*' teve o melhor AUC na validação cruzada.
    # Vamos usar o melhor modelo tunado da comparação inicial como fallback se outros falharam.
    if modelo_tunado_melhor:
       modelo_final_escolhido = modelo_tunado_melhor
       print(f"Modelo final escolhido: {type(modelo_final_escolhido).__name__} (tunado)")
    else:
       modelo_final_escolhido = melhor_modelo_comparacao
       print(f"Modelo final escolhido: {type(modelo_final_escolhido).__name__} (original da comparação)")

In [ ]:
# Finalizar o modelo escolhido (treina novamente com todos os dados de df_treino)
modelo_final = None
if modelo_final_escolhido:
    print(f"\nFinalizando o modelo {type(modelo_final_escolhido).__name__}...")
    modelo_final = finalize_model(modelo_final_escolhido)
    print("Modelo finalizado com sucesso.")
    print(modelo_final)
else:
    print("\nERRO: Nenhum modelo foi selecionado para finalização.")

print("\n" + "="*60 + "\n")

# Etapa 10: Avaliação no Conjunto de Validação (Hold-out)

In [ ]:
print("="*30)
print("ETAPA 10: AVALIAÇÃO NO CONJUNTO DE VALIDAÇÃO")
print("="*30)

In [ ]:
if modelo_final:
    print("Realizando predições no conjunto de validação (dados não vistos)...")
    # CORREÇÃO AQUI: Adicionar raw_score=True
    predicoes_validacao = predict_model(modelo_final, data=df_validacao, raw_score=True)

    print("\nPrimeiras linhas das predições na validação (com scores):")
    # Agora a coluna prediction_score_1 estará disponível
    print(predicoes_validacao[['Response', 'prediction_label', 'prediction_score_1']].head())
    # Calcular AUC e outras métricas no conjunto de validação
    try:
        from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score, f1_score, classification_report
        auc_validacao = roc_auc_score(predicoes_validacao['Response'], predicoes_validacao['prediction_score_1'])
        accuracy_validacao = accuracy_score(predicoes_validacao['Response'], predicoes_validacao['prediction_label'])
        precision_validacao = precision_score(predicoes_validacao['Response'], predicoes_validacao['prediction_label'])
        recall_validacao = recall_score(predicoes_validacao['Response'], predicoes_validacao['prediction_label'])
        f1_validacao = f1_score(predicoes_validacao['Response'], predicoes_validacao['prediction_label'])

        print(f"\nMétricas no Conjunto de Validação:")
        print(f"- AUC:      {auc_validacao:.4f}")
        print(f"- Acurácia: {accuracy_validacao:.4f}")
        print(f"- Precisão: {precision_validacao:.4f}")
        print(f"- Recall:   {recall_validacao:.4f}")
        print(f"- F1-Score: {f1_validacao:.4f}")
        print("\nRelatório de Classificação (Validação):")
        print(classification_report(predicoes_validacao['Response'], predicoes_validacao['prediction_label']))

    except Exception as e:
        print(f"\nErro ao calcular métricas de validação: {e}")
else:
    print("Não é possível avaliar no conjunto de validação, pois nenhum modelo foi finalizado.")

print("\n" + "="*60 + "\n")

# Etapa 11: Previsão nos Dados de Teste Finais (Xtest.csv)

In [ ]:
print("="*30)
print("ETAPA 11: PREVISÃO EM Xtest.csv")
print("="*30)
print("Aplicando as mesmas transformações de Limpeza e Engenharia de Atributos aos dados de Xtest...")

df_teste_final_transformado = df_teste_final.copy() # Usar o df_teste_final carregado no início

In [ ]:
# --- 11.1 Limpeza em Xtest.csv ---
# Remover colunas irrelevantes (verificar se existem em Xtest.csv)
colunas_para_remover_teste = ['ID', 'Z_CostContact', 'Z_Revenue', 'Dt_Customer', 'Year_Birth'] # Ajustar conforme Xtest.csv
colunas_existentes_para_remover = [col for col in colunas_para_remover_teste if col in df_teste_final_transformado.columns]
if colunas_existentes_para_remover:
    print(f"Removendo colunas de Xtest: {colunas_existentes_para_remover}")
    df_teste_final_transformado.drop(colunas_existentes_para_remover, axis=1, inplace=True, errors='ignore')

In [ ]:
# Tratar NaNs em 'Income' (usar a média do *conjunto de treino original* para consistência)
if 'Income' in df_teste_final_transformado.columns and df_teste_final_transformado['Income'].isna().any():
    print("Tratando NaNs em 'Income' para Xtest (usando média do treino)...")
    media_income_treino = df_treino_completo['Income'].mean() # Média do dataset original antes de dropna
    df_teste_final_transformado['Income'].fillna(media_income_treino, inplace=True)
else:
     print("Coluna 'Income' não encontrada ou sem NaNs em Xtest.")

In [ ]:
# Corrigir 'Marital_Status'
if 'Marital_Status' in df_teste_final_transformado.columns:
    print("Corrigindo 'Marital_Status' para Xtest...")
    df_teste_final_transformado['Marital_Status'] = df_teste_final_transformado['Marital_Status'].replace({'Alone': 'Single', 'Absurd': 'Single', 'YOLO': 'Single'})

In [ ]:
# --- 11.2 Ajustar Tipos de Dados em Xtest.csv ---
print("Ajustando tipos de dados para Xtest...")
df_teste_final_transformado = df_teste_final_transformado.convert_dtypes()
# Garantir tipos numéricos corretos
numeric_cols_teste = [col for col in df_teste_final_transformado.columns if df_teste_final_transformado[col].dtype in ['int64', 'float64', 'Int64', 'Float64']]
for col in numeric_cols_teste:
     # Verificar colunas numéricas chave
     if col in ['Age', 'Income', 'Time_Customer'] or 'Mnt' in col or 'Num' in col or 'AcceptedCmp' in col or 'Recency' in col:
         df_teste_final_transformado[col] = pd.to_numeric(df_teste_final_transformado[col], errors='coerce')
         # Re-imputar NaNs se a coerção falhar (improvável, mas seguro)
         if df_teste_final_transformado[col].isna().any():
             if col in df_treino.columns: # Usar média do df_treino (após limpeza)
                 media_treino = df_treino[col].mean()
                 print(f"Imputando NaNs (conversão) em '{col}' de Xtest com média do treino: {media_treino:.2f}")
                 df_teste_final_transformado[col].fillna(media_treino, inplace=True)
             elif col in df_treino_completo.columns: # Fallback para média original
                  media_treino_original = df_treino_completo[col].mean()
                  print(f"Aviso: Imputando NaNs (conversão) em '{col}' de Xtest com média *original* do treino: {media_treino_original:.2f}")
                  df_teste_final_transformado[col].fillna(media_treino_original, inplace=True)
             else: # Fallback para média do próprio teste (menos ideal)
                  media_teste = df_teste_final_transformado[col].mean()
                  print(f"Aviso Crítico: Imputando NaNs (conversão) em '{col}' de Xtest com média *do teste*: {media_teste:.2f}")
                  df_teste_final_transformado[col].fillna(media_teste, inplace=True)


In [ ]:
# --- 11.3 Engenharia de Atributos em Xtest.csv ---
print("Aplicando Engenharia de Atributos em Xtest...")
# 'Age' e 'Time_Customer' já devem existir e estar no formato correto em Xtest.csv (verificar arquivo)

In [ ]:
# Children_Total
if 'Kidhome' in df_teste_final_transformado.columns and 'Teenhome' in df_teste_final_transformado.columns:
    df_teste_final_transformado['Children_Total'] = df_teste_final_transformado['Kidhome'] + df_teste_final_transformado['Teenhome']
    print("- Feature 'Children_Total' criada para Xtest.")

In [ ]:
# Total Spending
colunas_mnt_teste = [col for col in df_teste_final_transformado.columns if 'Mnt' in col]
if colunas_mnt_teste:
    df_teste_final_transformado['Total_Spending'] = df_teste_final_transformado[colunas_mnt_teste].sum(axis=1)
    print("- Feature 'Total_Spending' criada para Xtest.")

In [ ]:
# Total Purchases
colunas_purch_teste = [col for col in df_teste_final_transformado.columns if 'Purchases' in col]
if colunas_purch_teste:
    df_teste_final_transformado['Total_Purchases'] = df_teste_final_transformado[colunas_purch_teste].sum(axis=1)
    print("- Feature 'Total_Purchases' criada para Xtest.")

In [ ]:
# Total_Acc_Cmp
if all(c in df_teste_final_transformado.columns for c in cmp_cols):
    df_teste_final_transformado['Total_Acc_Cmp'] = df_teste_final_transformado[cmp_cols].sum(axis=1)
    print("- Feature 'Total_Acc_Cmp' criada para Xtest.")
else:
    print("- Aviso: Colunas de AcceptedCmp não encontradas em Xtest, 'Total_Acc_Cmp' não criada.")

In [ ]:
# Income_Per_Capita
if 'Marital_Status' in df_teste_final_transformado.columns and 'Income' in df_teste_final_transformado.columns and 'Children_Total' in df_teste_final_transformado.columns:
    df_teste_final_transformado['Family_Size_Approx'] = df_teste_final_transformado['Marital_Status'].apply(family_size_approx)
    df_teste_final_transformado['Income_Per_Capita'] = df_teste_final_transformado['Income'] / (df_teste_final_transformado['Family_Size_Approx'] + df_teste_final_transformado['Children_Total'])
    df_teste_final_transformado['Income_Per_Capita'].replace([np.inf, -np.inf], 0, inplace=True)
    df_teste_final_transformado['Income_Per_Capita'].fillna(0, inplace=True)
    df_teste_final_transformado.drop('Family_Size_Approx', axis=1, inplace=True)
    print("- Feature 'Income_Per_Capita' criada para Xtest.")
else:
     print("- Aviso: Colunas necessárias para 'Income_Per_Capita' não encontradas em Xtest, feature não criada.")

In [ ]:
# Garantir que as colunas em df_teste_final_transformado correspondam às esperadas pelo pipeline
# (PyCaret's predict_model geralmente lida com isso, mas verificar é bom)
print("\nColunas em Xtest após transformações:", df_teste_final_transformado.columns.tolist())
print("\nCabeçalho de Xtest (após limpeza/FE):")
print(df_teste_final_transformado.head())
print("Shape de Xtest (após limpeza/FE):", df_teste_final_transformado.shape)

In [ ]:
#  --- 11.4 Realizar Predições ---
if modelo_final:
    print("\nRealizando predições em Xtest.csv...")
    # CORREÇÃO AQUI: Adicionar raw_score=True
    predicoes_teste_final = predict_model(modelo_final, data=df_teste_final_transformado, raw_score=True)
    print("Predições concluídas.")

    print("\nPrimeiras linhas das predições em Xtest.csv:")
    # Exibe 'prediction_label' (0 ou 1) e 'prediction_score_1' (probabilidade de ser 1)
    print(predicoes_teste_final[['prediction_label', 'prediction_score_1']].head())

    # Salvar as previsões (opcional)
    # try:
    #     predicoes_teste_final.to_csv('previsoes_xteste_final.csv', index=False)
    #     print("\nPredições de Xtest salvas em 'previsoes_xteste_final.csv'")
    # except Exception as e:
    #     print(f"\nErro ao salvar previsões: {e}")

else:
    print("Não é possível realizar predições em Xtest.csv, pois nenhum modelo foi finalizado.")

In [ ]:
print("\n" + "="*60)
print("FIM DO NOTEBOOK")
print("="*60 + "\n")

# Etapa 12: Salvar o Modelo Final (Opcional)

In [ ]:
# Descomente as linhas abaixo para salvar o modelo final treinado e o pipeline
print("="*30)
print("ETAPA 12: SALVAR MODELO FINAL (OPCIONAL)")
print("="*30)

In [ ]:
# Descomente as linhas abaixo para salvar o modelo final treinado e o pipeline
if modelo_final:
   try:
       save_model(modelo_final, 'modelo_ifood_final_refinado')
       print("\nModelo final salvo como 'modelo_ifood_final_refinado.pkl'")
       print("Este arquivo contém o pipeline de pré-processamento e o modelo treinado.")
   except Exception as e:
       print(f"\nErro ao salvar o modelo: {e}")
else:
   print("\nNenhum modelo final para salvar.")